In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
D_params = [D_W1, D_b1, D_W2, D_b2]

In [4]:
# Z is noised input
# G(z) => generated new x
Z = tf.placeholder(tf.float32, shape=[None, 100])
G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))
G_params = [G_W1, G_b1, G_W2, G_b2]

In [5]:
def discriminator(x):
    D_a1 = tf.matmul(x, D_W1) + D_b1
    D_h1 = tf.nn.relu(D_a1)
    # what is logit?
    # logit is inverse function of sigmoid function
    # therefore, D_logit = logit(D_prob).
    # 즉, sigmoid 함수에 들어가는 값이 logit 이라고 할 수 있음.
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    return D_prob, D_logit

In [6]:
def generator(z):
    G_a1 = tf.matmul(z, G_W1) + G_b1
    G_h1 = tf.nn.relu(G_a1)
    # what is differ between logit <> log_prob ?
    # 이것도 걍 로짓이 맞는거 같은데? logit_prob 의 약자일것 같음.
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    
    return G_prob

In [7]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [8]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [9]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# V1. paper-based learning (다른 코드를 보면 logit 을 이용해서 CE로 계산을 함. 이게 더 성능이 잘 나오나봄)
# http://bamos.github.io/2016/08/09/deep-completion/ 참고

# maximize x => minimize -x
#D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))
#G_loss = -tf.reduce_mean(tf.log(D_fake))

# V2.
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_params)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_params)

In [10]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
batch_size = 100
Z_dim = 100

In [12]:
D_losses = []
G_losses = []

In [13]:
sess = tf.Session()
# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

idx = 0

for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('out/{:0>4d}.png'.format(idx), bbox_inches='tight')
        idx += 1
        plt.close(fig)
    
    # _ 는 원래 Y 임.
    X_batch, _ = mnist.train.next_batch(batch_size)
    
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_batch, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})
    
    if it % 1000 == 0:
        print("[{:7d}] D_loss: {:.4f} / G_loss: {:.4f}".format(it, D_loss_curr, G_loss_curr))
        D_losses.append(D_loss_curr)
        G_losses.append(G_loss_curr)

[      0] D_loss: 1.3176 / G_loss: 2.3101
[   1000] D_loss: 0.0168 / G_loss: 10.1525
[   2000] D_loss: 0.0074 / G_loss: 8.2010
[   3000] D_loss: 0.0356 / G_loss: 5.4216
[   4000] D_loss: 0.0739 / G_loss: 5.7155
[   5000] D_loss: 0.2324 / G_loss: 6.0828
[   6000] D_loss: 0.2134 / G_loss: 4.8982
[   7000] D_loss: 0.3079 / G_loss: 5.2566
[   8000] D_loss: 0.4939 / G_loss: 4.1673
[   9000] D_loss: 0.5883 / G_loss: 3.3370
[  10000] D_loss: 0.4114 / G_loss: 3.7350
[  11000] D_loss: 0.4883 / G_loss: 2.7931
[  12000] D_loss: 0.4879 / G_loss: 2.6027
[  13000] D_loss: 0.5578 / G_loss: 2.7210
[  14000] D_loss: 0.7348 / G_loss: 2.4082
[  15000] D_loss: 0.5036 / G_loss: 2.3248
[  16000] D_loss: 0.5502 / G_loss: 2.5480
[  17000] D_loss: 0.6905 / G_loss: 2.1593
[  18000] D_loss: 0.7744 / G_loss: 1.9736
[  19000] D_loss: 0.7927 / G_loss: 2.1514
[  20000] D_loss: 0.5857 / G_loss: 1.9863
[  21000] D_loss: 0.6792 / G_loss: 2.0490
[  22000] D_loss: 0.6250 / G_loss: 2.2876
[  23000] D_loss: 0.6230 / G_loss